### To run Model all required code modules are marked with "*"

# Imports and Global Variables *

This notebook demonstrates various image enhancement techniques using OpenCV.

In [1]:
#imports
import cv2
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import model_selection
from pathlib import Path

import os
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


Dataset_path = "E:/Github/UTMIST-OpenCV-Image-Enchancement/Dataset4K"

## Testing that Data Exists and can be displayed

We will start by loading an image from the disk.

In [ ]:
image = cv2.imread("dataset4k/4k-3840-x-2160-wallpapers-themefoxx (1).jpg")

if image is None:
  print("image not found -- check to see if dataset path is correct or dataset is downloaded.")
else:
  print("image found")

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

half = cv2.resize(image, (0, 0), fx = 0.1, fy = 0.1)
plt.imshow(cv2.cvtColor(half, cv2.COLOR_BGR2RGB))
plt.title('Resized Image')
plt.show()

image found


## Storing and Creating a Training Dataset - i.e Downscaling Images - Test


In [ ]:
for each in Path(Dataset_path).iterdir():
  if each.is_file() and each.suffix in ['.jpg', '.jpeg', '.png']:
    image = cv2.imread(str(each))
    if image is not None:
      downscaled_image = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)
      output_path = Path("E:/Github/UTMIST-OpenCV-Image-Enchancement/Downscaled_Dataset") / each.name
      cv2.imwrite(str(output_path), downscaled_image)
      print(f"Saved downscaled image to {output_path}")
    else:
      print(f"Failed to read image {each}")

## Load Data into Tensors/Data Types - Decomissioned

In [20]:
def load_and_normalize_images(image_dir, normalize_range=(0, 1), target_size=(480, 270)):
    image_paths = list(Path(image_dir).glob('*'))
    images = []
    
    for image_path in image_paths:
        image = cv2.imread(str(image_path))
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, target_size)
            image = image.astype(np.float32)
            if normalize_range == (0, 1):
                image /= 255.0
            elif normalize_range == (-1, 1):
                image = (image / 127.5) - 1.0
            images.append(image)
        else:
            print(f"Failed to read image {image_path}")
    
    return np.array(images)

# Load and normalize images from Downscaled_Dataset
downscaled_images = load_and_normalize_images("E:/Github/UTMIST-OpenCV-Image-Enchancement/Downscaled_Dataset", normalize_range=(0, 1))
print(f"Loaded {len(downscaled_images)} images from Downscaled_Dataset")

Loaded 2056 images from Downscaled_Dataset


## Pre Loading Data *

In [2]:
class SuperResolutionDataset(Dataset):
    def __init__(self, hr_dir, transform=None, downscale_factor=2):
        """
        hr_dir: Directory containing the original 4K images.
        transform: A torchvision.transforms pipeline for image conversion.
        downscale_factor: Factor by which the image is downscaled to simulate low resolution.
        """
        self.hr_dir = hr_dir
        self.image_files = [os.path.join(hr_dir, file) for file in os.listdir(hr_dir)
                            if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform
        self.downscale_factor = downscale_factor

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        # Load the high-resolution (HR) image (target)
        hr_img = cv2.imread(self.image_files[index])
        if hr_img is None:
            raise ValueError(f"Image not found or cannot be read: {self.image_files[index]}")
        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
        h, w, _ = hr_img.shape

        # Create the low-resolution (LR) image:
        # 1. Downscale the HR image.
        lr_img = cv2.resize(hr_img, (w // self.downscale_factor, h // self.downscale_factor),
                            interpolation=cv2.INTER_CUBIC)
        # 2. Upscale back to original resolution.
        lr_img_upscaled = cv2.resize(lr_img, (w, h), interpolation=cv2.INTER_CUBIC)

        if self.transform:
            hr_img = self.transform(hr_img)
            lr_img_upscaled = self.transform(lr_img_upscaled)

        return lr_img_upscaled, hr_img

# Define transforms: convert images to tensors.
transform = transforms.Compose([
    transforms.ToTensor(),
    # You can add normalization here if needed.
])

# Specify your high-resolution images directory.
hr_images_dir = r"E:/Github/UTMIST-OpenCV-Image-Enchancement/Dataset4K"
dataset = SuperResolutionDataset(hr_dir=hr_images_dir, transform=transform, downscale_factor=2)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2)


## Model Architecture for Residual Blocks and 3x3 Conv. v ReLu Layer*

In [3]:
# Residual Block: 3x3 Conv -> ReLU -> BatchNorm -> 3x3 Conv -> BatchNorm with skip connection.
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu  = nn.ReLU(inplace=True)
        self.bn1   = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(channels)

    def forward(self, x):
        identity = x  # Save input for the skip connection.
        out = self.conv1(x)
        out = self.relu(out)
        out = self.bn1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity  # Skip connection
        out = self.relu(out)
        return out

# Super-resolution network using residual blocks and PixelShuffle for upsampling.
class SuperResolutionNet(nn.Module):
    def __init__(self, num_channels=3, num_features=64, num_res_blocks=5, upscale_factor=2):
        """
        num_channels: Number of channels in the image (3 for RGB).
        num_features: Number of features after the initial convolution.
        num_res_blocks: Number of residual blocks.
        upscale_factor: Factor to upscale the input image.
        """
        super(SuperResolutionNet, self).__init__()
        
        # Step 1: Feature Extraction
        self.initial_conv = nn.Conv2d(num_channels, num_features, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        
        # Step 2: Residual Blocks
        res_blocks = []
        for _ in range(num_res_blocks):
            res_blocks.append(ResidualBlock(num_features))
        self.res_blocks = nn.Sequential(*res_blocks)
        
        # Step 3: Upsampling using sub-pixel convolution (PixelShuffle)
        # For PixelShuffle, output channels of the conv must equal (upscale_factor^2 * num_features)
        self.upsample_conv = nn.Conv2d(num_features, num_features * (upscale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)
        
        # Step 4: Post-processing: Final convolution to refine output.
        self.final_conv = nn.Conv2d(num_features, num_channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        # Extract low-level features.
        out = self.initial_conv(x)
        out = self.relu(out)
        
        # Residual blocks for deep feature extraction.
        out = self.res_blocks(out)
        
        # Upsampling.
        out = self.upsample_conv(out)
        out = self.pixel_shuffle(out)
        
        # Final refinement.
        out = self.final_conv(out)
        return out

In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SuperResolutionNet(num_channels=3, num_features=64, num_res_blocks=5, upscale_factor=2).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10  # Set epochs as needed
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for lr_imgs, hr_imgs in dataloader:
        lr_imgs = lr_imgs.to(device)
        hr_imgs = hr_imgs.to(device)
        
        optimizer.zero_grad()
        outputs = model(lr_imgs)
        loss = criterion(outputs, hr_imgs)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f}")
    
    # (Optional) You could also calculate PSNR and SSIM here to monitor image quality.


NameError: name 'torch' is not defined

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
    print("Using GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Using CPU.")


: 